# 自動機械学習 Automated Machine Learning による離反予測モデリング & モデル解釈 (リモート高速実行)


顧客属性データを用いて離反予測モデルを開発します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 計算環境の準備
- 自動機械学習の事前設定
- モデル学習と結果の確認
- モデル解釈

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-classification-churn-remote'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning service の Dataset のフォーマットでデータを定義します。

In [4]:
data = "https://amlgitsamples.blob.core.windows.net/churn/CATelcoCustomerChurnTrainingSample.csv"
dataset = Dataset.Tabular.from_delimited_files(data)

In [5]:
dataset.take(5).to_pandas_dataframe()

,age,annualincome,calldroprate,callfailurerate,callingnum,customerid,customersuspended,education,gender,homeowner,...,totalminsusedinlastmonth,unpaidbalance,usesinternetservice,usesvoiceservice,percentagecalloutsidenetwork,totalcallduration,avgcallduration,churn,year,month
0,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,5971,663,0,2015,1
1,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,3981,995,0,2015,2
2,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,7379,737,0,2015,1
3,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,1729,432,0,2015,2
4,58,27076,0.07,0.02,4251055773,3,True,Master or equivalent,Female,True,...,216,144,False,False,0.48,3122,624,0,2015,1


In [6]:
train_dataset, test_dataset = dataset.random_split(0.8, seed=1234)

In [7]:
X_train = train_dataset.drop_columns(columns=['churn'])
y_train = train_dataset.keep_columns(columns=['churn'], validate=True)
X_test  = test_dataset.drop_columns(columns=['churn'])
y_test  = test_dataset.keep_columns(columns=['churn'], validate=True)

### 計算環境 (Machine Learning Compute) の設定

In [8]:
# 予め cpucluster という名称の Machine Learning Compute を作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(ws, "cpucluster")

In [9]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True

cd = CondaDependencies.create(conda_packages=['numpy','py-xgboost<=0.80', 'tensorflow>=1.10.0,<=1.12.0'])
conda_run_config.environment.python.conda_dependencies = cd

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [10]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 50,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             X = X_train,
                             y = y_train,
                             run_configuration=conda_run_config,
                             max_concurrent_iterations = 10,
                             **automl_settings
                            )

### 実行と結果確認

In [11]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote compute: cpucluster
Parent Run ID: AutoML_070370e8-7d94-4276-a952-035bc896e274
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         9   StandardScalerWrapper XGBoostClassifier        0:03:42       0.7745    0.7745
         8   MaxAbsScaler SGD                               0:04:28       0.6622    0.7745
         7   MaxAbsScaler ExtremeRand

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

In [13]:
# 詳細ログの出力
remote_run.get_details()

{'runId': 'AutoML_070370e8-7d94-4276-a952-035bc896e274',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-10-02T16:25:52.408424Z',
 'endTimeUtc': '2019-10-02T16:45:27.614937Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'MaxTimeSeconds': '300',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'cpucluster',
  'RawAMLSettingsString': "{'name': 'automl-classification-churn-remote', 'path': None, 'subscription_id': '9c0f91b8-eb2f-484c-979c-15848c098a6b', 'resource_group': 'mlservice', 'workspace_name': 'azureml', 'region': 'eastus', 'compute_target': 'cpucluster', 'spark_service': None, 'iterations': 50, 'primary_metric': 'AUC_weighted', 'task_type': 'classification', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 2, 'y_min': None, 'y_max': None, 'num_classes': None, 'preprocess': True, 'lag_length': 0

In [14]:
best_run, fitted_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-classification-churn-remote,AutoML_070370e8-7d94-4276-a952-035bc896e274_44,azureml.scriptrun,Completed,Link to Azure Portal,Link to Documentation


### モデルの理解

In [15]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'age',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'annualincome',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'calldroprate',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'callfailurerate',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'callingnum',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'customerid',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'monthlybilledamount',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
 

In [16]:
from pprint import pprint


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()


print_model(fitted_model)

datatransformer
{'enable_feature_sweeping': None,
 'feature_sweeping_timeout': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing.data',
 'with_mean': False,
 'with_std': False}

XGBoostClassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'eta': 0.3,
 'gamma': 0,
 'grow_policy': 'lossguide',
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_delta_step': 0,
 'max_depth': 9,
 'max_leaves': 31,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 600,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'reg:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1.875,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.8,
 'tree_method': 'hist',
 'verbose': -10}



## 3. Automated ML Explainer

In [17]:
from azureml.train.automl.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations

automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, X=X_train, X_test=X_test, y=y_train, task='classification')

Current status: Setting up data for AutoMl explanations
Current status: Setting up the AutoML featurization for explanations
Current status: Setting up the AutoML estimator
Current status: Setting up the AutoML featurizer
Current status: Generating a feature map for raw feature importance
Current status: Finding all classes from the dataset
Current status: Data for AutoMl explanations successfully setup


In [18]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

Using older than supported version of lightgbm, please upgrade to version greater than 2.2.1


In [19]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)
#print(raw_explanations.get_feature_importance_dict())

In [20]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
ExplanationDashboard(raw_explanations, automl_explainer_setup_obj.automl_pipeline, automl_explainer_setup_obj.X_test_raw)

ExplanationWidget(value={'predictedY': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0…